# Gold layer

Gold = DW (modelo dimensional): publica dimensões e fatos com nomes de negócio (`gold.dim_*`, `gold.fact_*`) e chaves/prefixos corretos (`sk_`, `nk_`, `fk_`).

In [0]:
USE CATALOG data_modelling;
CREATE SCHEMA IF NOT EXISTS gold;

## 1) gold.dim_tempo

In [0]:
-- ============================================
-- GOLD: dim_tempo (INSERT-ONLY, idempotente)
-- ============================================
-- Range de datas derivado da Silver
CREATE OR REPLACE TEMP VIEW tempo_range AS
SELECT
  COALESCE(MIN(order_date), DATE('2019-01-01')) AS dt_min,
  COALESCE(MAX(order_date), DATE('2030-12-31')) AS dt_max
FROM silver.orders_clean;

-- Série diária
CREATE OR REPLACE TEMP VIEW tempo_series AS
WITH r AS (SELECT dt_min, dt_max FROM tempo_range),
seq AS (SELECT sequence(dt_min, dt_max, INTERVAL 1 DAY) AS dts FROM r)
SELECT explode(dts) AS data FROM seq;

-- Mapeamentos estáveis (sem depender de locale)
CREATE OR REPLACE TEMP VIEW dim_tempo_stage AS
WITH base AS (
  SELECT
    CAST(date_format(data, 'yyyyMMdd') AS INT) AS tempo_sk,
    data,
    YEAR(data)  AS ano,
    QUARTER(data) AS trimestre,
    MONTH(data) AS mes,
    DAY(data)   AS dia,
    CASE WHEN dayofweek(data)=1 THEN 7 ELSE dayofweek(data)-1 END AS dia_semana  -- 1=Seg ... 7=Dom
  FROM tempo_series
)
SELECT
  tempo_sk,
  CAST(date_format(data, 'yyyyMMdd') AS INT) AS date_id,
  data,
  ano,
  trimestre,
  mes,
  dia,
  dia_semana,
  -- nomes PT-BR fixos por mapeamento
  element_at(map(
    1,'janeiro', 2,'fevereiro', 3,'março', 4,'abril', 5,'maio', 6,'junho',
    7,'julho', 8,'agosto', 9,'setembro', 10,'outubro', 11,'novembro', 12,'dezembro'
  ), mes) AS nome_mes,
  element_at(map(
    1,'segunda', 2,'terça', 3,'quarta', 4,'quinta', 5,'sexta', 6,'sábado', 7,'domingo'
  ), dia_semana) AS nome_dia
FROM base;

-- Tabela alvo
CREATE TABLE IF NOT EXISTS gold.dim_tempo (
  tempo_sk   INT,
  date_id    INT,          -- yyyymmdd
  data       DATE,
  ano        INT,
  trimestre  INT,
  mes        INT,
  dia        INT,
  dia_semana INT,          -- 1=Seg ... 7=Dom
  nome_mes   STRING,
  nome_dia   STRING
) USING DELTA;

-- MERGE INSERT-ONLY (não faz UPDATE nunca)
MERGE INTO gold.dim_tempo t
USING dim_tempo_stage s
ON t.tempo_sk = s.tempo_sk
WHEN NOT MATCHED THEN INSERT (
  tempo_sk, date_id, data, ano, trimestre, mes, dia, dia_semana, nome_mes, nome_dia
) VALUES (
  s.tempo_sk, s.date_id, s.data, s.ano, s.trimestre, s.mes, s.dia, s.dia_semana, s.nome_mes, s.nome_dia
);

## 2) gold.dim_produto

In [0]:
-- ============================================
-- GOLD: dim_produto (NK = product_id)
-- ============================================
CREATE TABLE IF NOT EXISTS gold.dim_produto (
  product_sk    BIGINT GENERATED ALWAYS AS IDENTITY,
  nk_product_id STRING,
  product_name  STRING,
  category      STRING,
  subcategory   STRING,
  brand         STRING,
  cost_price    DECIMAL(18,2),
  list_price    DECIMAL(18,2),
  is_active     STRING,
  last_update   TIMESTAMP,
  row_hash      STRING
) USING DELTA;

-- Stage com hash (reuso da Silver já “clean”)
CREATE OR REPLACE TEMP VIEW dim_produto_stage AS
SELECT
  product_id      AS nk_product_id,
  product_name,
  category,
  subcategory,
  brand,
  cost_price,
  list_price,
  is_active,
  last_update,
  sha2(concat_ws('||',
    coalesce(product_name,''),
    coalesce(category,''),
    coalesce(subcategory,''),
    coalesce(brand,''),
    cast(coalesce(cost_price,0) as string),
    cast(coalesce(list_price,0) as string),
    coalesce(is_active,''),
    coalesce(date_format(last_update,'yyyy-MM-dd HH:mm:ss'),'')
  ),256) AS row_hash
FROM silver.products_clean;

MERGE INTO gold.dim_produto t
USING dim_produto_stage s
ON t.nk_product_id = s.nk_product_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.product_name = s.product_name,
  t.category     = s.category,
  t.subcategory  = s.subcategory,
  t.brand        = s.brand,
  t.cost_price   = s.cost_price,
  t.list_price   = s.list_price,
  t.is_active    = s.is_active,
  t.last_update  = s.last_update,
  t.row_hash     = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  nk_product_id, product_name, category, subcategory, brand,
  cost_price, list_price, is_active, last_update, row_hash
) VALUES (
  s.nk_product_id, s.product_name, s.category, s.subcategory, s.brand,
  s.cost_price, s.list_price, s.is_active, s.last_update, s.row_hash
);

## 3) gold.dim_cliente

In [0]:
%sql
-- DROP TABLE gold.dim_cliente

In [0]:
-- ============================================================
-- GOLD: dim_cliente (snapshot corrente, idempotente)
-- ============================================================
-- 1) Tabela alvo (1 linha por NK; com row_hash para idempotência)
CREATE TABLE IF NOT EXISTS gold.dim_cliente (
  customer_sk      BIGINT,          -- SK atual da SCD (muda quando abre nova versão)
  nk_customer_id   STRING,          -- NK estável do cliente (chave do MERGE)
  customer_name    STRING,
  email            STRING,
  city             STRING,
  state            STRING,
  effective_start  TIMESTAMP,
  effective_end    TIMESTAMP,
  row_hash         STRING
) USING DELTA;

-- Backfill do hash se necessário (evita updates "em massa" na 1ª execução após adicionar a coluna)
UPDATE gold.dim_cliente
SET row_hash = COALESCE(row_hash,
  sha2(concat_ws('||',
    coalesce(customer_name,''),
    coalesce(lower(email),''),
    coalesce(trim(city),''),
    coalesce(upper(trim(state)),''),
    coalesce(date_format(effective_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(effective_end,  'yyyy-MM-dd HH:mm:ss'),'')
  ),256)
);

-- 2) Stage: pega SOMENTE o corrente da SCD, normaliza e calcula hash
CREATE OR REPLACE TEMP VIEW dim_cliente_stage AS
SELECT
  s.customer_sk,
  s.customer_id                                      AS nk_customer_id,
  s.customer_name,
  lower(s.email)                                     AS email,
  trim(s.city)                                       AS city,
  upper(trim(s.state))                               AS state,
  s.effective_start,
  s.effective_end,
  sha2(concat_ws('||',
    coalesce(s.customer_name,''),
    coalesce(lower(s.email),''),
    coalesce(trim(s.city),''),
    coalesce(upper(trim(s.state)),''),
    coalesce(date_format(s.effective_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(s.effective_end,  'yyyy-MM-dd HH:mm:ss'),'')
  ),256) AS row_hash
FROM silver.dim_customer_scd s
WHERE s.is_current = TRUE;

-- 3) MERGE idempotente (1 linha por nk_customer_id)
MERGE INTO gold.dim_cliente t
USING dim_cliente_stage s
ON t.nk_customer_id = s.nk_customer_id                 -- snapshot corrente = 1 por NK
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN
  UPDATE SET
    t.customer_sk     = s.customer_sk,                 -- atualiza SK quando a SCD virar
    t.customer_name   = s.customer_name,
    t.email           = s.email,
    t.city            = s.city,
    t.state           = s.state,
    t.effective_start = s.effective_start,
    t.effective_end   = s.effective_end,
    t.row_hash        = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  customer_sk, nk_customer_id, customer_name, email, city, state, effective_start, effective_end, row_hash
) VALUES (
  s.customer_sk, s.nk_customer_id, s.customer_name, s.email, s.city, s.state, s.effective_start, s.effective_end, s.row_hash
);

-- 4) (Opcional) Linha "desconhecida" para FKs nulas
INSERT INTO gold.dim_cliente
SELECT 0, '_UNKNOWN_', 'Desconhecido', NULL, NULL, NULL, TIMESTAMP('1900-01-01'), TIMESTAMP('9999-12-31'),
       sha2('_UNKNOWN_',256)
WHERE NOT EXISTS (SELECT 1 FROM gold.dim_cliente WHERE nk_customer_id = '_UNKNOWN_');

In [0]:
-- ============================================
-- GOLD: dim_cliente_scd (replicação da SCD da Silver)
-- ============================================
CREATE SCHEMA IF NOT EXISTS gold;

CREATE TABLE IF NOT EXISTS gold.dim_cliente_scd (
  customer_sk      BIGINT,
  customer_id      STRING,
  customer_name    STRING,
  email            STRING,
  city             STRING,
  state            STRING,
  effective_start  TIMESTAMP,
  effective_end    TIMESTAMP,
  is_current       BOOLEAN,
  row_hash         STRING
) USING DELTA;

-- Stage com hash para idempotência
CREATE OR REPLACE TEMP VIEW dim_cliente_scd_stage AS
SELECT
  s.customer_sk,
  s.customer_id,
  s.customer_name,
  s.email,
  s.city,
  s.state,
  s.effective_start,
  s.effective_end,
  s.is_current,
  sha2(concat_ws('||',
    coalesce(s.customer_name,''),
    coalesce(lower(s.email),''),
    coalesce(trim(s.city),''),
    coalesce(upper(trim(s.state)),''),
    coalesce(date_format(s.effective_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(s.effective_end,  'yyyy-MM-dd HH:mm:ss'),''),
    cast(coalesce(s.is_current,false) as string)
  ),256) AS row_hash
FROM silver.dim_customer_scd s;

MERGE INTO gold.dim_cliente_scd t
USING dim_cliente_scd_stage s
ON t.customer_sk = s.customer_sk
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.customer_id      = s.customer_id,
  t.customer_name    = s.customer_name,
  t.email            = s.email,
  t.city             = s.city,
  t.state            = s.state,
  t.effective_start  = s.effective_start,
  t.effective_end    = s.effective_end,
  t.is_current       = s.is_current,
  t.row_hash         = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  customer_sk, customer_id, customer_name, email, city, state,
  effective_start, effective_end, is_current, row_hash
) VALUES (
  s.customer_sk, s.customer_id, s.customer_name, s.email, s.city, s.state,
  s.effective_start, s.effective_end, s.is_current, s.row_hash
);

## 4) gold.fact_vendas

In [0]:
-- ============================================
-- GOLD: fact_vendas_current (sem time-aware)
-- ============================================

CREATE TABLE IF NOT EXISTS gold.fact_vendas_current (
  order_id        STRING,
  order_item_id   STRING,
  fk_tempo_sk     INT,
  fk_customer_sk  BIGINT,
  fk_product_sk   BIGINT,
  quantity        DECIMAL(18,2),
  unit_price      DECIMAL(18,2),
  discount_amount DECIMAL(18,2),
  revenue_gross   DECIMAL(18,2),
  revenue_net     DECIMAL(18,2),
  row_hash        STRING
) USING DELTA
PARTITIONED BY (fk_tempo_sk);

CREATE OR REPLACE TEMP VIEW fact_snapshot_stage AS
WITH oi AS (
  SELECT order_item_id, order_id, product_id, quantity, unit_price, discount_amount
  FROM silver.order_items_clean
),
o AS (
  SELECT order_id, customer_id, order_date
  FROM silver.orders_clean
),
dt AS (
  SELECT tempo_sk, data
  FROM gold.dim_tempo
),
dp AS (
  SELECT nk_product_id, product_sk
  FROM gold.dim_produto
),
c_snap AS (
  SELECT nk_customer_id, customer_sk
  FROM gold.dim_cliente  -- snapshot corrente
),
base AS (
  SELECT
    oi.order_item_id,
    oi.order_id,
    o.customer_id,
    o.order_date,
    oi.product_id,
    oi.quantity,
    oi.unit_price,
    oi.discount_amount,
    dt.tempo_sk              AS fk_tempo_sk,
    dp.product_sk            AS fk_product_sk,
    COALESCE(c.customer_sk, 0) AS fk_customer_sk
  FROM oi
  JOIN o ON o.order_id = oi.order_id
  JOIN dt ON dt.data = o.order_date
  LEFT JOIN dp ON dp.nk_product_id = oi.product_id
  LEFT JOIN c_snap c ON c.nk_customer_id = o.customer_id
  WHERE o.order_date >= date_sub(current_date(), 60)
)
SELECT
  order_id,
  order_item_id,
  fk_tempo_sk,
  fk_customer_sk,
  fk_product_sk,
  quantity,
  unit_price,
  discount_amount,
  CAST(quantity * unit_price AS DECIMAL(18,2)) AS revenue_gross,
  CAST(quantity * unit_price - COALESCE(discount_amount,0) AS DECIMAL(18,2)) AS revenue_net,
  sha2(concat_ws('||',
    order_id, order_item_id,
    cast(coalesce(fk_tempo_sk,0) as string),
    cast(coalesce(fk_customer_sk,0) as string),
    cast(coalesce(fk_product_sk,0) as string),
    cast(coalesce(quantity,0) as string),
    cast(coalesce(unit_price,0) as string),
    cast(coalesce(discount_amount,0) as string),
    cast(coalesce(quantity * unit_price,0) as string)
  ),256) AS row_hash
FROM base;

MERGE INTO gold.fact_vendas_current t
USING fact_snapshot_stage s
ON  t.order_item_id = s.order_item_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.order_id        = s.order_id,
  t.fk_tempo_sk     = s.fk_tempo_sk,
  t.fk_customer_sk  = s.fk_customer_sk,
  t.fk_product_sk   = s.fk_product_sk,
  t.quantity        = s.quantity,
  t.unit_price      = s.unit_price,
  t.discount_amount = s.discount_amount,
  t.revenue_gross   = s.revenue_gross,
  t.revenue_net     = s.revenue_net,
  t.row_hash        = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  order_id, order_item_id, fk_tempo_sk, fk_customer_sk, fk_product_sk,
  quantity, unit_price, discount_amount, revenue_gross, revenue_net, row_hash
) VALUES (
  s.order_id, s.order_item_id, s.fk_tempo_sk, s.fk_customer_sk, s.fk_product_sk,
  s.quantity, s.unit_price, s.discount_amount, s.revenue_gross, s.revenue_net, s.row_hash
);

In [0]:
%sql
-- DROP TABLE gold.fact_vendas

In [0]:
%sql
-- ============================================
-- GOLD: fact_vendas (grão = item de pedido)
-- Chaves via dimensões GOLD (inclui cliente SCD time-aware)
-- ============================================
-- Tabela alvo (particionada por tempo)
CREATE TABLE IF NOT EXISTS gold.fact_vendas (
  order_id        STRING,
  order_item_id   STRING,
  fk_tempo_sk     INT,
  fk_customer_sk  BIGINT,
  fk_product_sk   BIGINT,
  quantity        DECIMAL(18,2),
  unit_price      DECIMAL(18,2),
  discount_amount DECIMAL(18,2),
  revenue_gross   DECIMAL(18,2),  -- quantity * unit_price
  revenue_net     DECIMAL(18,2),  -- revenue_gross - discount_amount
  row_hash        STRING
) USING DELTA
PARTITIONED BY (fk_tempo_sk);

-- =========================
-- Stage (60d de watermark)
-- =========================
CREATE OR REPLACE TEMP VIEW fact_stage AS
WITH
oi AS (   -- Itens do pedido (Silver curado)
  SELECT
    order_item_id,
    order_id,
    product_id,
    quantity,
    unit_price,
    discount_amount,
    updated_at
  FROM silver.order_items_clean
),
o AS (    -- Pedidos (Silver curado)
  SELECT
    order_id,
    customer_id,
    order_date,
    order_status,
    total_amount
  FROM silver.orders_clean
),
dt AS (   -- Dim tempo (Gold)
  SELECT tempo_sk, data
  FROM gold.dim_tempo
),
dp AS (   -- Dim produto (Gold)
  SELECT nk_product_id, product_sk
  FROM gold.dim_produto
),
c_scd AS ( -- Dim cliente SCD (Gold) para range join
  SELECT
    customer_sk,
    customer_id,
    CAST(effective_start AS DATE) AS eff_start_date,
    CAST(effective_end   AS DATE) AS eff_end_date
  FROM gold.dim_cliente_scd
),
base AS (
  SELECT
    oi.order_item_id,
    oi.order_id,
    o.customer_id,
    o.order_date,
    oi.product_id,
    oi.quantity,
    oi.unit_price,
    oi.discount_amount,
    -- lookups de dimensões (apenas GOLD)
    d.tempo_sk              AS fk_tempo_sk,
    p.product_sk            AS fk_product_sk,
    COALESCE(c.customer_sk, 0) AS fk_customer_sk  -- fallback unknown
  FROM oi
  JOIN o  ON o.order_id = oi.order_id
  JOIN dt d ON d.data   = o.order_date
  LEFT JOIN dp p ON p.nk_product_id = oi.product_id
  LEFT JOIN c_scd c
    ON c.customer_id   = o.customer_id
   AND o.order_date   >= c.eff_start_date
   AND o.order_date   <= c.eff_end_date
  WHERE o.order_date >= date_sub(current_date(), 60)  -- watermark (ajuste conforme sua latência)
)
SELECT
  order_id,
  order_item_id,
  fk_tempo_sk,
  COALESCE(fk_customer_sk, 0) AS fk_customer_sk,
  fk_product_sk,
  quantity,
  unit_price,
  discount_amount,
  CAST(quantity * unit_price AS DECIMAL(18,2))                            AS revenue_gross,
  CAST(quantity * unit_price - COALESCE(discount_amount,0) AS DECIMAL(18,2)) AS revenue_net,
  sha2(concat_ws('||',
    order_id, order_item_id,
    cast(coalesce(fk_tempo_sk,0) as string),
    cast(coalesce(fk_customer_sk,0) as string),
    cast(coalesce(fk_product_sk,0) as string),
    cast(coalesce(quantity,0) as string),
    cast(coalesce(unit_price,0) as string),
    cast(coalesce(discount_amount,0) as string),
    cast(coalesce(quantity * unit_price,0) as string)
  ),256) AS row_hash
FROM base;

-- =========================
-- MERGE idempotente
-- =========================
MERGE INTO gold.fact_vendas t
USING fact_stage s
ON  t.order_item_id = s.order_item_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.order_id        = s.order_id,
  t.fk_tempo_sk     = s.fk_tempo_sk,
  t.fk_customer_sk  = s.fk_customer_sk,
  t.fk_product_sk   = s.fk_product_sk,
  t.quantity        = s.quantity,
  t.unit_price      = s.unit_price,
  t.discount_amount = s.discount_amount,
  t.revenue_gross   = s.revenue_gross,
  t.revenue_net     = s.revenue_net,
  t.row_hash        = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  order_id, order_item_id, fk_tempo_sk, fk_customer_sk, fk_product_sk,
  quantity, unit_price, discount_amount, revenue_gross, revenue_net, row_hash
) VALUES (
  s.order_id, s.order_item_id, s.fk_tempo_sk, s.fk_customer_sk, s.fk_product_sk,
  s.quantity, s.unit_price, s.discount_amount, s.revenue_gross, s.revenue_net, s.row_hash
);


## 5) Views de conveniência

In [0]:
%sql
CREATE OR REPLACE VIEW gold.vw_vendas_diarias AS
SELECT
  dt.data,
  SUM(f.revenue_net) AS receita_liquida,
  SUM(f.quantity)    AS itens
FROM gold.fact_vendas f
JOIN gold.dim_tempo dt ON dt.tempo_sk = f.fk_tempo_sk
GROUP BY dt.data;

CREATE OR REPLACE VIEW gold.vw_receita_por_estado AS
SELECT
  dt.data,
  dc.state,
  SUM(f.revenue_net) AS receita_liquida
FROM gold.fact_vendas f
JOIN gold.dim_tempo   dt ON dt.tempo_sk    = f.fk_tempo_sk
JOIN gold.dim_cliente dc ON dc.customer_sk = f.fk_customer_sk
GROUP BY dt.data, dc.state;
